In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [80]:
from szp_funcs import load_month_all, print_df, custom_create_res, to_double, day_types, path_to_docs

In [3]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp_2021/'
months = ['mar', 'apr', 'may', 'jun', 'jul', 'aug']

In [74]:
trans = {'mar': 'март', 'apr': 'апрель','may': 'май', 'jun': 'июнь', 'jul': 'июль', 'aug': 'август' }

In [4]:
path_to_data

'C:/Users/PetukhovMD/Desktop/szp_2021/'

In [90]:
schools = pd.read_excel(path_to_docs + 'all_schools.xlsx')

In [94]:
t_sch = schools[['Unnamed: 8', 'Unnamed: 3']].tail(677)

In [95]:
d = {key: value for (key, value) in zip(t_sch['Unnamed: 8'].to_list(), t_sch['Unnamed: 3'].to_list())}

In [81]:
snils_of_int = pd.read_excel(path_to_docs + 'cpm_ppl.xlsx')

In [83]:
snils_of_int = snils_of_int['snils'].to_list()

In [56]:
ans = 0
for month in months:
    t = pd.read_excel(path_to_data + month + '.xlsx')
    t['date'] = month
    t = t[['snils', 'inn', 'date', 'type']][(t.type == day_types[0]) | (t.type == day_types[-1])]
    if isinstance(ans, pd.DataFrame) == False:
        ans = t
        continue
    ans = pd.concat([ans, t])

In [57]:
ans = ans.drop_duplicates()

In [118]:
sort = ans[['snils', 'inn']][ans.snils.isin(snils_of_int)].drop_duplicates().groupby('snils').agg(['unique'])

In [159]:
def give_compex_names(row, name):
    s = row[name].tolist()
    ans = ''
    for inn in s:
        ans += d[inn] + '|'
    return ans[:-1]

In [162]:
sort['name'] = sort.apply(lambda row: give_compex_names(row, 'inn'), axis=1)

In [163]:
sort

,inn,name
snils,,
001-172-924 91,[7725618950],ГАОУ ДПО ЦПМ
001-181-742 88,[7725618950],ГАОУ ДПО ЦПМ
001-192-334 85,[7725618950],ГАОУ ДПО ЦПМ
001-369-806 20,"[7725618950, 7727270740]","ГАОУ ДПО ЦПМ|ГБПОУ ОК ""Юго-Запад"""
001-416-633 87,[7725618950],ГАОУ ДПО ЦПМ
...,...,...
203-460-950 29,[7725618950],ГАОУ ДПО ЦПМ
203-541-437 17,[7725618950],ГАОУ ДПО ЦПМ
203-803-813 27,[7725618950],ГАОУ ДПО ЦПМ


In [166]:
print_df(sort[['name']], 'Места работы цпм')

In [143]:
give_compex_names(sort[sort.index == '001-172-924 91'], 'inn')

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U3')) -> None

In [126]:
sort

,inn
snils,
001-172-924 91,[7725618950]
001-181-742 88,[7725618950]
001-192-334 85,[7725618950]
001-369-806 20,"[7725618950, 7727270740]"
001-416-633 87,[7725618950]
...,...
203-460-950 29,[7725618950]
203-541-437 17,[7725618950]
203-803-813 27,[7725618950]


In [123]:
sort.columns = ['inn']

In [125]:
sort[len(sort.inn) > 1]

KeyError: True

In [60]:
t_ans = ans.groupby(['snils', 'date', 'type']).max().unstack().unstack()

In [62]:
t_ans.columns = [s[1] + '_' + s[2] for s in t_ans.columns]

In [72]:
t_ans = t_ans.reset_index()

In [65]:
restruct_ans = pd.DataFrame()

In [68]:
t_ans.index

Index(['001-001-208 25', '001-004-622 47', '001-005-047 42', '001-005-586 64',
       '001-005-644 57', '001-005-786 70', '001-008-669 78', '001-010-312 25',
       '001-011-996 67', '001-017-453 65',
       ...
       '214-028-137 11', '214-106-146 00', '214-166-236 31', '214-205-407 04',
       '214-284-865 62', '214-322-883 32', '214-412-929 30', '214-417-077 31',
       '214-505-689 46', '214-556-778 75'],
      dtype='object', name='snils', length=177883)

In [77]:
restruct_ans = t_ans[['snils'] + [day_types[0] + '_' + s for s in months] + [day_types[2] + '_' + s for s in months]]

In [98]:
def give_name(row, name):
    if pd.isna(row[name]):
        return np.nan
    return d[row[name]]

In [89]:
out_ans = ans[ans.snils.isin(snils_of_int)]

In [101]:
out_ans['inn'] = out_ans.apply(lambda row: give_name(row, 'inn'), axis=1)

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_22560\1041462143.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_ans['inn'] = out_ans.apply(lambda row: give_name(row, 'inn'), axis=1)


In [88]:
out_restruct_ans = restruct_ans[restruct_ans.snils.isin(snils_of_int)]

In [104]:
for month in months:
    out_restruct_ans[day_types[0] + '_' + month] = out_restruct_ans.apply(lambda row: give_name(row, day_types[0] + '_' + month), axis=1)
    out_restruct_ans[day_types[2] + '_' + month] = out_restruct_ans.apply(lambda row: give_name(row, day_types[2] + '_' + month), axis=1)

C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_22560\119929675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_restruct_ans[day_types[0] + '_' + month] = out_restruct_ans.apply(lambda row: give_name(row, day_types[0] + '_' + month), axis=1)
C:\Users\PetukhovMD\AppData\Local\Temp\ipykernel_22560\119929675.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_restruct_ans[day_types[2] + '_' + month] = out_restruct_ans.apply(lambda row: give_name(row, day_types[2] + '_' + month), axis=1)
C:\Users\P

In [105]:
out_restruct_ans

,snils,Основное место работы_mar,Основное место работы_apr,Основное место работы_may,Основное место работы_jun,Основное место работы_jul,Основное место работы_aug,Внешнее совместительство_mar,Внешнее совместительство_apr,Внешнее совместительство_may,Внешнее совместительство_jun,Внешнее совместительство_jul,Внешнее совместительство_aug
315,001-172-924 91,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,NaN,ГАОУ ДПО ЦПМ,NaN,NaN,NaN,NaN,NaN,NaN
368,001-181-742 88,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,NaN,NaN,NaN,NaN,NaN,NaN
512,001-192-334 85,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,NaN,NaN,NaN,NaN,NaN,NaN
2050,001-369-806 20,"ГБПОУ ОК ""Юго-Запад""","ГБПОУ ОК ""Юго-Запад""","ГБПОУ ОК ""Юго-Запад""","ГБПОУ ОК ""Юго-Запад""","ГБПОУ ОК ""Юго-Запад""","ГБПОУ ОК ""Юго-Запад""",ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,NaN,ГАОУ ДПО ЦПМ
2419,001-416-633 87,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,NaN,ГАОУ ДПО ЦПМ,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
176956,203-460-950 29,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,NaN,NaN,NaN,NaN,NaN,NaN
176976,203-541-437 17,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,NaN,NaN,NaN,NaN,NaN,NaN
177020,203-803-813 27,NaN,NaN,NaN,NaN,NaN,ГАОУ ДПО ЦПМ,NaN,NaN,NaN,NaN,NaN,NaN
177102,204-310-658 04,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,ГАОУ ДПО ЦПМ,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
print_df(out_restruct_ans, 'ЦПМ по снилс')
print_df(out_ans, 'ЦПМ все уникальные')

In [64]:
for s in t_ans:
    print(s)

Внешнее совместительство_apr
Внешнее совместительство_aug
Внешнее совместительство_jul
Внешнее совместительство_jun
Внешнее совместительство_mar
Внешнее совместительство_may
Основное место работы_apr
Основное место работы_aug
Основное место работы_jul
Основное место работы_jun
Основное место работы_mar
Основное место работы_may


In [8]:
load_month_all('jun', path_to_data = path_to_data)

,inn,snils,sum_jun,job_jun,day_jun,type_jun,stv_jun,status_jun,status_pref_jun,wday_jun,nday_jun
0,5003021368,001-191-281 85,135785.86,Учитель,NaN,Основное место работы,1.0,Отпуск основной,Работа,18.0,21.0
1,5003021368,001-244-107 65,103703.39,Учитель,NaN,Основное место работы,1.0,Отпуск основной,Работа,16.0,21.0
2,5003021368,001-263-234 77,18560.00,Воспитатель,NaN,Основное место работы,1.0,Работа,Отпуск основной,7.0,21.0
3,5003021368,001-287-587 24,52000.00,Помощник воспитателя,NaN,Основное место работы,1.0,Работа,Работа,21.0,21.0
4,5003021368,001-322-594 80,78780.95,Старший кладовщик,NaN,Основное место работы,1.0,Работа,Работа,22.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...
173683,9718071371,196-882-391 45,56412.95,Бухгалтер,NaN,Основное место работы,1.0,NaN,Работа,16.0,21.0
173684,9718071371,198-731-766 35,50000.00,Учитель,NaN,Основное место работы,1.0,Работа,Работа,NaN,NaN
173685,9718071371,199-418-719 35,120866.57,Бухгалтер,NaN,Основное место работы,1.0,Работа,Работа,21.0,21.0
173686,9718071371,205-412-630 13,307676.33,Начальник группы,NaN,Основное место работы,1.0,Работа,Работа,20.0,21.0


In [8]:
res = custom_create_res(months, load_month_all)

KeyboardInterrupt: 